In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
import numpy as np

import os
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

#plotting
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff


import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [2]:
data_folder = r'data'

In [3]:
#creating folder to save the backttests
backtest_folder = r'Backtest'
if not os.path.exists(backtest_folder):
    os.makedirs(backtest_folder)

In [4]:
#making parameters grid
parameters_grid = []
for entry_momentum in range(-10,-2):
    for exit_momentum in range(entry_momentum+1, -1,): #this is also entry_pnl
        for exit_pnl in range(exit_momentum+1,11):
            parameters_grid.append((entry_momentum,exit_momentum,exit_pnl))

In [5]:
#trade params
df_symbols = pd.read_csv(r'df_symbols.csv', encoding='utf-8')

In [6]:
#df_symbols['Product'].unique()

In [7]:
#df_symbols.head(2)

# Outliers

In [68]:
df_all = pd.DataFrame()
df_outliers = pd.DataFrame()
for filename in [x for x in os.listdir(data_folder) if '.csv' in x]:
    ticker = filename.replace('.csv','')
    
     #trade exchange information
    single_point_value = df_symbols[df_symbols['Symbol']==ticker]['Single point value']\
                     .values[0]
    ticks = df_symbols[df_symbols['Symbol']==ticker]['Ticks']\
                     .values[0]
    settlement_time = pd.to_datetime(df_symbols[df_symbols['Symbol']==ticker]['Settlement']\
                     .values[0]).time()
    
    #importing price history
    df = pd.read_csv(data_folder+r'\\'+filename, encoding='utf-8').drop('Unnamed: 0',1)
    df['timestamp'] = df[['date','time']].apply(lambda x:\
                          pd.to_datetime(x[0]+x[1].split('days')[1]), axis=1) 
    df['date'] = df['timestamp'].dt.date
    df['time'] = df['time'].apply(lambda x: pd.to_datetime(x.split('days')[1].strip()).time())
        
    df = df.sort_values(by = 'timestamp')
    df = df.reset_index(drop=True)
    
    #MAKING A DF, DATE AS AN INDEX PRICES AS COLUMNS
    #QUESTIONS: IT SEEMS THAT THE BAR CLOSE IS NOT EQUAL TO NEXT BAR OPEN, IS THIS APPROACH CORRECT?
    df_features = pd.DataFrame()

    for time in df['time'].unique():
        minute_change = (dt.datetime.combine(dt.date.today(), time) \
                    - dt.datetime.combine(dt.date.today(), settlement_time)).seconds/60-1
        if minute_change>=24*60-20:
            minute_change = minute_change-24*60

        minute_change = str(minute_change).split('.')[0]
        if minute_change != '9':

            df_part = df[df['time']==time].set_index('date')[['open_p']].copy()
            df_part.columns = ['price_'+minute_change]
        else:
            df_part = df[df['time']==time].set_index('date')[['open_p','close_p']].copy()
            df_part.columns = ['price_'+minute_change,'price_10']

        df_features = df_features.join(df_part, how='outer')
    
    df_min = pd.DataFrame(df_features.reset_index().groupby('date').min().min(axis=1))
    df_min.columns = ['min']
    df_max = pd.DataFrame(df_features.reset_index().groupby('date').max().max(axis=1))
    df_max.columns = ['max']
    df_features = df_min.join(df_max)
    df_features['ticker'] = ticker
    
    df_features['momentum'] = (df_features['max']/df_features['min']-1)*100
    df_outliers = pd.concat([df_outliers,df_features[df_features['momentum']>1.75].copy()])
    df_features = df_features.sort_values(by='momentum').iloc[-5:]
    df_features['momentum_std'] = (df_features['momentum']-df_features['momentum'].mean())/df_features['momentum'].std()
    df_all = pd.concat([df_all,df_features.reset_index()])
#     df_outliers = pd.concat([df_outliers,\
#     pd.DataFrame(df_features.sort_values(by='momentum').iloc[-1]).transpose().reset_index()])

In [69]:
df_outliers.shape

(382, 4)

In [70]:
df_outliers['ticker'].nunique()

28

In [71]:
df_outliers.groupby('ticker')['ticker'].count().sort_values().tail(10)

ticker
GAS#    16
QHO#    16
QNG#    21
@HE#    21
QBZ#    24
EB#     26
IRB#    29
QRB#    37
CRD#    43
QCL#    44
Name: ticker, dtype: int64

In [72]:
df_outliers = df_outliers.reset_index()[['ticker','date']]

outliers_dict = {}
for ticker in df_outliers['ticker'].unique():
    outliers_dict[ticker] = list(df_outliers[df_outliers['ticker']==ticker]['date'].values)

In [75]:
outliers_dict.keys()

dict_keys(['@CC#', '@CT#', '@ES#', '@GF#', '@HE#', '@KC#', '@LE#', '@MFS#', '@MME#', '@NQ#', '@RTY#', '@SB#', '@SM#', '@UB#', '@YM#', 'CRD#', 'EB#', 'GAS#', 'IHO#', 'IRB#', 'QBZ#', 'QCL#', 'QHO#', 'QNG#', 'QPA#', 'QPL#', 'QRB#', 'QSI#'])

# Checking

# Loop

In [94]:
for filename in [x for x in os.listdir(data_folder) if '.csv' in x]:
    ticker = filename.replace('.csv','')
    
        
    
     #trade exchange information
    single_point_value = df_symbols[df_symbols['Symbol']==ticker]['Single point value']\
                     .values[0]
    ticks = df_symbols[df_symbols['Symbol']==ticker]['Ticks']\
                     .values[0]
    settlement_time = pd.to_datetime(df_symbols[df_symbols['Symbol']==ticker]['Settlement']\
                     .values[0]).time()
    
    #importing price history
    df = pd.read_csv(data_folder+r'\\'+filename, encoding='utf-8').drop('Unnamed: 0',1)
    df['timestamp'] = df[['date','time']].apply(lambda x:\
                          pd.to_datetime(x[0]+x[1].split('days')[1]), axis=1) 
    df['date'] = df['timestamp'].dt.date
    
    #dropping outliers
    if ticker in outliers_dict.keys():
        outlier_dates = outliers_dict[ticker]
        df = df[~df['date'].isin(outlier_dates)].copy()
    
    df['time'] = df['time'].apply(lambda x: pd.to_datetime(x.split('days')[1].strip()).time())
        
    df = df.sort_values(by = 'timestamp')
    df = df.reset_index(drop=True)
    
    #MAKING A DF, DATE AS AN INDEX PRICES AS COLUMNS
    #QUESTIONS: IT SEEMS THAT THE BAR CLOSE IS NOT EQUAL TO NEXT BAR OPEN, IS THIS APPROACH CORRECT?
    df_features = pd.DataFrame()

    for time in df['time'].unique():
        minute_change = (dt.datetime.combine(dt.date.today(), time) \
                    - dt.datetime.combine(dt.date.today(), settlement_time)).seconds/60-1
        if minute_change>=24*60-20:
            minute_change = minute_change-24*60

        minute_change = str(minute_change).split('.')[0]
        if minute_change != '9':

            df_part = df[df['time']==time].set_index('date')[['open_p']].copy()
            df_part.columns = ['price_'+minute_change]
        else:
            df_part = df[df['time']==time].set_index('date')[['open_p','close_p']].copy()
            df_part.columns = ['price_'+minute_change,'price_10']

        df_features = df_features.join(df_part, how='outer')
     
    #RUNNING BACKTEST AND STORING RESULTS
    results = pd.DataFrame()
    for parameters in parameters_grid:
        entry_momentum = 'price_'+str(parameters[0])
        exit_momentum = 'price_'+str(parameters[1])
        exit_pnl = 'price_'+str(parameters[2])
        
        if (entry_momentum in df_features.columns) & \
           (exit_momentum in df_features.columns) & \
           (exit_pnl in df_features.columns): 
            #calculating momentum
            df_features['momentum'] = (df_features[exit_momentum]/df_features[entry_momentum]-1)*100
            i=0
            for momentum_threshold in \
            df_features['momentum'].quantile([0.1,0.2,0.3,0.6,0.7,0.8,0.9]).values:
                i = i+1
                #print(i)
                if momentum_threshold>0:
                    #go long
                    df_features['pnl'] = df_features[['momentum',exit_momentum,exit_pnl]].\
                                         apply(lambda x: x[2]-x[1] if x[0]>momentum_threshold else np.nan, axis=1)\
                                         *single_point_value
                else:
                    #go short
                    df_features['pnl'] = df_features[['momentum',exit_momentum,exit_pnl]].\
                                         apply(lambda x: x[1]-x[2] if x[0]<momentum_threshold else np.nan, axis=1)\
                                         *single_point_value

                #calculating stats
                if df_features.dropna(subset=['pnl']).shape[0]>0:
                    summary = {}
                    summary['entry_momentum'] = str(parameters[0])
                    summary['exit_momentum'] = str(parameters[1])
                    summary['exit_pnl'] = str(parameters[2])
                    summary['momentum_threshold'] = momentum_threshold
                    summary['total_days'] = df_features.shape[0]
                    summary['days_traded'] = df_features[pd.notnull(df_features['pnl'])].shape[0]
                    summary['days_traded %'] = summary['days_traded']/summary['total_days']
                    summary['total_pnl'] = df_features['pnl'].sum()
                    summary['mean_pnl'] = df_features['pnl'].mean()
                    summary['max pnl'] = df_features['pnl'].max()
                    summary['max pnl date'] = df_features.sort_values(by='pnl').dropna(subset=['pnl']).index[-1]
                    summary['min pnl'] = df_features['pnl'].min()
                    summary['min pnl date'] = str(df_features.sort_values(by='pnl').index[0])
                    summary['std pnl'] = df_features['pnl'].std()
                    summary['sharpe_ratio'] = (df_features['pnl'].mean()/df_features['pnl'].std())*(252**0.5)
                    summary['Win %'] = df_features[df_features['pnl']>0].shape[0]/df_features[pd.notnull(df_features['pnl'])].shape[0]
                    summary['# of Winners'] = df_features[df_features['pnl']>0].shape[0]
                    summary['mean_pnl_winners'] = df_features[df_features['pnl']>=0]['pnl'].mean()
                    summary['mean_pnl_losers'] = df_features[df_features['pnl']<0]['pnl'].mean()
                    summary['avg $ move'] = np.mean(df_features['pnl']/single_point_value)
                    summary['avg tick move'] = np.mean(df_features['pnl']/ticks)
                    results = pd.concat([results,pd.DataFrame(summary, index=[0])])
    results['ticker'] = ticker
    results.sort_values(by='sharpe_ratio').to_csv(\
    backtest_folder+r'\\'+ticker+'.csv', encoding='utf-8', index=False)

In [91]:
df.head(30)

,date,time,open_p,high_p,low_p,close_p,tot_vlm,prd_vlm,num_trds,timestamp
0,2018-01-02,13:51:00,146.350,146.375,146.350,146.375,5445,10,0,2018-01-02 13:51:00
1,2018-01-02,13:52:00,146.375,146.400,146.350,146.375,5468,22,0,2018-01-02 13:52:00
2,2018-01-02,13:53:00,146.375,146.425,146.325,146.425,5496,23,0,2018-01-02 13:53:00
3,2018-01-02,13:54:00,146.450,146.450,146.300,146.300,5526,28,0,2018-01-02 13:54:00
4,2018-01-02,13:55:00,146.325,146.475,146.300,146.475,5555,22,0,2018-01-02 13:55:00
5,2018-01-02,13:56:00,146.475,146.575,146.475,146.475,5667,103,0,2018-01-02 13:56:00
6,2018-01-02,13:57:00,146.500,146.600,146.475,146.525,5712,30,0,2018-01-02 13:57:00
7,2018-01-02,13:58:00,146.525,146.525,146.475,146.475,5750,31,0,2018-01-02 13:58:00
8,2018-01-02,13:59:00,146.475,146.500,146.475,146.475,5765,13,0,2018-01-02 13:59:00
9,2018-01-02,14:00:00,146.500,147.175,146.275,147.175,6619,774,0,2018-01-02 14:00:00


In [87]:
df_features

,price_-10,price_-9,price_-8,price_-7,price_-6,price_-5,price_-4,price_-3,price_-2,price_-1,price_0,price_1,price_2,price_3,price_4,price_5,momentum,pnl
date,,,,,,,,,,,,,,,,,,
2018-01-02,33.500,33.500,33.490,33.520,33.510,33.520,33.520,33.540,33.530,33.550,33.560,33.550,33.540,33.570,33.570,nan,0.000,nan
2018-01-03,33.870,33.870,33.870,33.870,33.880,33.880,33.880,33.890,33.880,33.890,33.910,33.890,33.870,33.880,33.870,nan,0.000,nan
2018-01-04,33.840,33.850,33.860,33.860,33.850,33.850,33.860,33.870,33.840,33.870,33.860,33.880,33.880,33.870,33.870,nan,0.030,nan
2018-01-05,33.790,33.780,33.750,33.740,33.730,33.730,33.710,33.750,33.750,33.760,33.760,33.750,33.740,33.740,33.750,nan,-0.030,nan
2018-01-08,33.480,33.490,33.450,33.470,33.480,33.510,33.520,33.540,33.550,33.550,33.550,33.550,33.550,33.540,33.550,nan,0.030,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-09,53.040,53.040,53.060,53.050,53.000,53.000,52.970,52.920,52.810,52.790,52.780,52.840,52.800,52.770,52.750,nan,0.000,nan
2021-04-12,51.810,51.730,51.700,51.740,51.700,51.730,51.640,51.600,51.610,51.500,51.400,51.520,51.570,51.530,51.510,nan,-0.154,nan
2021-04-13,52.970,52.990,53.080,53.140,53.140,53.060,52.980,52.940,52.940,52.920,52.990,53.030,53.020,53.020,52.980,nan,0.038,nan


In [86]:
#to check
print(entry_momentum,exit_momentum,exit_pnl, ticker)

price_-10 price_-9 price_6 @BO#


In [83]:
#to check
print(entry_momentum,exit_momentum,exit_pnl, ticker)

price_-10 price_-9 price_5 @BO#


In [ ]:
entry_momentum = 'price_'+str(parameters[0])
exit_momentum = 'price_'+str(parameters[1])
exit_pnl = 'price_'+str(parameters[2])

In [81]:
df_features

,price_-10,price_-9,price_-8,price_-7,price_-6,price_-5,price_-4,price_-3,price_-2,price_-1,price_0,price_1,price_2,price_3,price_4,price_5,momentum,pnl
date,,,,,,,,,,,,,,,,,,
2018-01-02,33.500,33.500,33.490,33.520,33.510,33.520,33.520,33.540,33.530,33.550,33.560,33.550,33.540,33.570,33.570,nan,0.000,nan
2018-01-03,33.870,33.870,33.870,33.870,33.880,33.880,33.880,33.890,33.880,33.890,33.910,33.890,33.870,33.880,33.870,nan,0.000,nan
2018-01-04,33.840,33.850,33.860,33.860,33.850,33.850,33.860,33.870,33.840,33.870,33.860,33.880,33.880,33.870,33.870,nan,0.030,nan
2018-01-05,33.790,33.780,33.750,33.740,33.730,33.730,33.710,33.750,33.750,33.760,33.760,33.750,33.740,33.740,33.750,nan,-0.030,nan
2018-01-08,33.480,33.490,33.450,33.470,33.480,33.510,33.520,33.540,33.550,33.550,33.550,33.550,33.550,33.540,33.550,nan,0.030,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-09,53.040,53.040,53.060,53.050,53.000,53.000,52.970,52.920,52.810,52.790,52.780,52.840,52.800,52.770,52.750,nan,0.000,nan
2021-04-12,51.810,51.730,51.700,51.740,51.700,51.730,51.640,51.600,51.610,51.500,51.400,51.520,51.570,51.530,51.510,nan,-0.154,nan
2021-04-13,52.970,52.990,53.080,53.140,53.140,53.060,52.980,52.940,52.940,52.920,52.990,53.030,53.020,53.020,52.980,nan,0.038,nan
